In [5]:
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

def line(x, a, b):
    '''
    y = ax + b
    '''
    return a * x + b

def line_1(y, a, b):
    '''
    as y = ax + b
    '''
    return ( y - b ) / a

## Расчёт давления изопропанола при 298K

In [9]:
p = np.array([1, 10, 40, 100, 760]) #mmHg
T = np.array([-26.1, 2.4, 23.8, 39.5, 82.5]) #Celsius
#Convert units of p and T
T = T + 273.15 #K
p = p*0.001333224*10**5 #bar

...

In [10]:
p_isopr = 0.01051 #bar

## Расчёт ТД величин по получившейся константе (промежуточные резы в табличке)

In [12]:
K_p_res = np.array([ 0.0860, 0.1131, 0.1368])
T = np.array([140, 150, 160])
T = T + 273.15

In [18]:
d_r_H = linregress(1000/T, -np.log(K_p_res))[0]*8.314

In [19]:
linregress(1000/T, -np.log(K_p_res))

LinregressResult(slope=4.158446680726365, intercept=-7.6236439515868675, rvalue=0.9959470162622742, pvalue=0.0573362576281739, stderr=0.37554054943678433, intercept_stderr=0.8879839769202583)